# Logistic Regression with SGD Momentum

### Importing necessary modules

In [19]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.10, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /home/timur/code/ml_practice/logistic_regression_scratch
plugins: remotedata-0.3.1, openfiles-0.3.2, doctestplus-0.3.0, arraydiff-0.3
collected 5 items                                                              

test_function.py ....                                                    [ 80%]
test_models.py F                                                         [100%]

=================================== FAILURES ===================================
_________________________________ test_softmax _________________________________

    def test_softmax():
        N = 25
        d = 14
>       a = np.random.randn((N,d)).reshape(3,-1)

test_models.py:8: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
mtrand.pyx:1220: in numpy.random.mtrand.RandomState.randn
    ???
mtrand.pyx:1377: in numpy.rando

In [1]:
import numpy as np

from src.models import LogisticRegressionBinary
from src.models import LogisticRegressionMulti

from src.preprocessing import *

from src.evaluation import *

### Loading train/test datasets

In [2]:
X_train, y_train = load_data("train")

X_train = X_train.reshape(-1,28*28) # flatten images to get vector
y_train = y_train.reshape(-1,1)

# +1 - even
# -1 - odd
#y_train_parity = np.where(y_train%2, -1, 1) 

# Standardize train dataset
scaler = Scaler()
scaler.fit(X_train)
X_train = scaler.standardize(X_train)

In [4]:
X_test, y_test = load_data("t10k")

In [5]:
X_test_scaled = scaler.standardize(X_test.reshape(-1, 28 * 28))
y_test_parity = np.where(y_test % 2, -1, 1) 

### Random Search

In [5]:
params = {"C" : [0.3, 1, 2],
          "learning_rate_init" : [10**(-x) for x in np.arange(1,5,1, dtype="float")],
          "weight_decay" : np.arange(6, 9 , 1) * 0.1,
          "alpha" : np.arange(6, 9 , 1) * 0.1
}

### PCA

In [4]:
X_train = PCAimpl(X_train, 100)

In [6]:
X_train = Polynomial(X_train)

In [7]:
model = LogisticRegressionMulti(num_classes=8)

In [8]:
model.fit(X_train, y_train)

MemoryError: Unable to allocate 1.78 GiB for an array with shape (47335, 5051) and data type float64

In [10]:
from sklearn.metrics import accuracy_score
preds = model.predict(X_train)
accuracy_score(preds, y_train)

0.9702968205344882